# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [5]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
import ollama

In [6]:
# Initialize and constants

# load_dotenv(override=True)
# api_key = os.getenv('OPENAI_API_KEY')

# if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
#     print("API key looks good so far")
# else:
#     print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
# MODEL = 'gpt-4o-mini'
# openai = OpenAI()

import os
import requests
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

OLLAMA_MODEL = 'llama3.2'  # or any other Ollama model you've pulled
OLLAMA_API_URL = os.getenv("OLLAMA_API_URL", "http://localhost:11434")

# Optional: Check if Ollama server is running
try:
    response = requests.get(f"{OLLAMA_API_URL}/api/tags")
    if response.status_code == 200:
        print(f"Ollama server is running. Available models: {[m['name'] for m in response.json().get('models', [])]}")
    else:
        print("Ollama server is not responding as expected.")
except requests.exceptions.ConnectionError:
    print("Cannot connect to Ollama server. Is it running on localhost:11434?")

# Set your model
MODEL = OLLAMA_MODEL


Ollama server is running. Available models: ['llama3.2:latest']


In [7]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [9]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [10]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [11]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [12]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [13]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [17]:
# def get_links(url):
#     website = Website(url)
#     response = openai.chat.completions.create(
#         model=MODEL,
#         messages=[
#             {"role": "system", "content": link_system_prompt},
#             {"role": "user", "content": get_links_user_prompt(website)}
#       ],
#         response_format={"type": "json_object"}
#     )
#     result = response.choices[0].message.content
#     return json.loads(result)

import requests
import json

def get_links(url):
    website = Website(url)

    # Construct prompt manually (since Ollama does not support roles like OpenAI)
    prompt = link_system_prompt.strip() + "\n\n" + get_links_user_prompt(website).strip()

    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": MODEL,
            "prompt": prompt,
            "stream": False
        }
    )
    
    response.raise_for_status()
    result = response.json().get("response", "")
    
    # Attempt to parse the JSON result
    try:
        return json.loads(result)
    except json.JSONDecodeError:
        print("Failed to parse JSON from model response:")
        print(result)
        return {}


In [29]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://anthropic.com")
huggingface.links

['#main',
 '#footer',
 'https://www.anthropic.com/',
 'https://www.anthropic.com/claude',
 'https://www.anthropic.com/max',
 'https://www.anthropic.com/team',
 'https://www.anthropic.com/enterprise',
 'https://www.anthropic.com/pricing',
 'https://claude.ai/download',
 'https://claude.ai/',
 'https://www.anthropic.com/news/claude-character',
 'https://www.anthropic.com/api',
 'https://docs.anthropic.com/',
 'https://www.anthropic.com/pricing#api',
 'https://console.anthropic.com/',
 'https://docs.anthropic.com/en/docs/welcome',
 'https://www.anthropic.com/solutions/agents',
 'https://www.anthropic.com/solutions/coding',
 'https://www.anthropic.com/solutions/customer-support',
 'https://www.anthropic.com/solutions/education',
 'https://www.anthropic.com/solutions/financial-services',
 'https://www.anthropic.com/customers',
 'https://www.anthropic.com/research',
 'https://www.anthropic.com/economic-index',
 'https://www.anthropic.com/claude/opus',
 'https://www.anthropic.com/claude/sonne

In [30]:
get_links("https://anthropic.com")

{'links': [{'type': 'company page', 'url': 'https://www.anthropic.com/'},
  {'type': 'about the founder', 'url': 'https://www.anthropic.com/claude'},
  {'type': 'company page', 'url': 'https://www.anthropic.com/max'},
  {'type': 'team page', 'url': 'https://www.anthropic.com/team'},
  {'type': 'enterprise page', 'url': 'https://www.anthropic.com/enterprise'},
  {'type': 'pricing page', 'url': 'https://www.anthropic.com/pricing'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'},
  {'type': 'economic index page',
   'url': 'https://www.anthropic.com/economic-index'},
  {'type': "about the founder's poetry",
   'url': 'https://www.anthropic.com/claude/opus'},
  {'type': "about the founder's poetry",
   'url': 'https://www.anthropic.com/claude/sonnet'},
  {'type': "about the founder's poetry",
   'url': 'https://www.anthropic.com/claude/haiku'},
  {'type': 'about AI safety and responsibility',
   'url': 'https://www.anthropic.com/news/core-views-on-ai-safety'},
  {

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [31]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [33]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/'}, {'type': 'home page', 'url': 'https://www.anthropic.com/'}, {'type': 'company page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions'}, {'type': 'engineering page', 'url': 'https://www.anthropic.com/engineering'}, {'type': 'enterprise page', 'url': 'https://www.anthropic.com/enterprise'}, {'type': 'pricing page', 'url': 'https://www.anthropic.com/pricing'}, {'type': 'blog page', 'url': 'https://www.anthropic.com/blog'}, {'type': 'products page', 'url': 'https://docs.anthropic.com/'}, {'type': 'download center', 'url': 'https://claude.ai/download'}]}
Landing page:
Webpage

In [35]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [36]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

In [38]:
# def create_brochure(company_name, url):
#     response = openai.chat.completions.create(
#         model=MODEL,
#         messages=[
#             {"role": "system", "content": system_prompt},
#             {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
#           ],
#     )
#     result = response.choices[0].message.content
#     display(Markdown(result))

import requests
from IPython.display import Markdown, display

# Define the system prompt
system_prompt = """
You are a creative marketing assistant. Create a professional, engaging, and concise company brochure in **Markdown** format. Include sections such as:
- Company Overview
- Products/Services
- Mission & Vision
- Contact Information

Use headings, bullet points, and short paragraphs for readability.
"""

# Define the user prompt builder
def get_brochure_user_prompt(company_name, url):
    return f"The company name is **{company_name}**. The website is: {url}. Please generate a company brochure based on this."

# Main function using Ollama
def create_brochure(company_name, url):
    prompt = system_prompt.strip() + "\n\n" + get_brochure_user_prompt(company_name, url).strip()

    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": MODEL,  # Make sure MODEL is set to something like "llama3"
            "prompt": prompt,
            "stream": False
        }
    )

    response.raise_for_status()
    result = response.json().get("response", "")
    display(Markdown(result))

# Example call (assuming Ollama is running)
MODEL = "llama3.2"
create_brochure("Anthropic", "https://Anthropic.com")


# Anthropic Brochure
=====================

## Company Overview
-----------------

At Anthropic, we're on a mission to transform the way AI is developed and used. Our team of experts is dedicated to creating cutting-edge solutions that bridge the gap between technology and humanity.

We believe that the future of AI should be designed with ethics, transparency, and accountability in mind. Our goal is to empower developers, researchers, and organizations to build more responsible and inclusive AI systems.

## Products/Services
------------------

### **Anthropic AI Platform**

Our flagship product is an open-source AI platform designed for building, deploying, and managing AI models. The platform offers a range of tools and features, including:

*   Model training and deployment
*   Hyperparameter tuning
*   Model monitoring and tracking
*   Integration with popular frameworks

### **AI-as-a-Service (AIA)**

Anthropic's AIA offering provides access to our AI platform on-demand. With AIA, users can focus on developing their applications without worrying about the underlying infrastructure.

### **Consulting Services**

Our team of experts offers consulting services to help organizations integrate Anthropic's solutions into their existing workflows.

## Mission & Vision
-----------------

### **Mission Statement**

At Anthropic, our mission is to harness the power of AI to create a better world for all. We're committed to:

*   Developing responsible and inclusive AI systems
*   Empowering developers and researchers with cutting-edge tools
*   Fostering a community that promotes transparency and accountability

### **Vision Statement**

Our vision is to revolutionize the way AI is developed, used, and governed. We aim to create an ecosystem that prioritizes human values and ethics in AI development.

## Contact Information
-------------------

Ready to learn more about Anthropic's solutions or explore our products? Get in touch with us today!

*   Website: [https://Anthropic.com](https://Anthropic.com)
*   Email: \[info@anthropic.com](mailto:info@anthropic.com)
*   Phone: +1 (123) 456-7890
*   Address: 123 Main St, San Francisco, CA 94105

In [39]:
create_brochure("Anthropic", "https://Anthropic.com")

# Anthropic Brochure
==========================

### Company Overview

At Anthropic, we're passionate about harnessing the power of artificial intelligence to drive human progress. Our team of experts combines cutting-edge research with practical expertise to deliver innovative solutions that transform industries and improve lives.

### Products/Services
--------------------

*   **AI Solutions**: We develop tailored AI solutions for businesses across various sectors, including healthcare, finance, and education.
*   **Data Analytics**: Our data analytics platform helps organizations uncover insights and make informed decisions.
*   **Research & Development**: We collaborate with top researchers to push the boundaries of AI and create new technologies.

### Mission & Vision
--------------------

Our mission is to empower humans through technology, while ensuring that our solutions prioritize transparency, accountability, and social responsibility. Our vision is a world where AI enhances human capabilities, fosters creativity, and drives positive change.

*   **Empower Humans**: We believe in harnessing the power of AI to improve people's lives.
*   **Collaborate & Innovate**: We partner with experts from diverse backgrounds to create innovative solutions.
*   **Lead by Example**: We strive to set a high standard for responsible AI development and deployment.

### Contact Information
---------------------

Ready to learn more about Anthropic's cutting-edge AI solutions? Visit our website at <https://Anthropic.com> or get in touch with us:

*   Email: [info@anthropic.com](mailto:info@anthropic.com)
*   Phone: +1 555 123 4567
*   Address: 123 Main St, San Francisco, CA 94105

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [45]:
# def stream_brochure(company_name, url):
#     stream = openai.chat.completions.create(
#         model=MODEL,
#         messages=[
#             {"role": "system", "content": system_prompt},
#             {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
#           ],
#         stream=True
#     )
    
#     response = ""
#     display_handle = display(Markdown(""), display_id=True)
#     for chunk in stream:
#         response += chunk.choices[0].delta.content or ''
#         response = response.replace("```","").replace("markdown", "")
#         update_display(Markdown(response), display_id=display_handle.display_id)

import requests
import json
from IPython.display import display, Markdown, update_display

def stream_brochure(company_name, url):
    # Build the prompt
    prompt = system_prompt.strip() + "\n\n" + get_brochure_user_prompt(company_name, url).strip()

    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": MODEL,
            "prompt": prompt,
            "stream": True
        },
        stream=True
    )

    output = ""
    display_handle = display(Markdown(""), display_id=True)

    for line in response.iter_lines():
        if line:
            data = line.decode("utf-8").strip()
            if data.startswith("data:"):
                data = data[5:].strip()
            try:
                chunk = json.loads(data)
                new_content = chunk.get("response", "")
                output += new_content
                output = output.replace("```", "").replace("markdown", "")
                update_display(Markdown(output), display_id=display_handle.display_id)
            except json.JSONDecodeError:
                continue  # skip malformed lines

# ✅ Call the function like this
stream_brochure("Anthropic", "https://anthropic.com")



# Anthropic Brochure
======================

## Company Overview
-----------------

Welcome to Anthropic, your partner in AI innovation. We're a team of expert researchers and engineers dedicated to advancing the field of artificial intelligence.

At Anthropic, we believe that AI has the potential to transform industries and improve lives. Our mission is to develop cutting-edge AI solutions that are both effective and responsible.

## Products/Services
---------------------

### Research & Development

* **AI Model Optimization**: Our team specializes in optimizing complex machine learning models for improved performance and efficiency.
* **Data Annotation**: We provide high-quality data annotation services to help businesses prepare their data for AI model training.
* **Custom AI Solutions**: Our expert researchers work closely with clients to develop tailored AI solutions that meet specific business needs.

### Enterprise Products

* **Anthropic Platform**: Our cloud-based platform provides a suite of tools and resources for building, deploying, and managing AI models.
* **AI Model Serving**: We offer fast and secure model serving solutions for businesses looking to integrate AI into their operations.

## Mission & Vision
-------------------

Our mission at Anthropic is to harness the power of artificial intelligence to create a better future for all. We strive to:

* **Advance AI Research**: Push the boundaries of what's possible with AI through ongoing research and development.
* **Deliver Value to Customers**: Provide innovative AI solutions that drive business success and improve lives.
* **Foster Collaboration**: Encourage open dialogue between academia, industry, and government to advance AI responsibly.

Our vision is a future where AI augmentation enables humans to thrive in an increasingly complex world. We're committed to making this vision a reality through our work at Anthropic.

## Contact Information
---------------------

Stay connected with us:

* **Website**: https://anthropic.com
* **Email**: [info@anthropic.com](mailto:info@anthropic.com)
* **Social Media**:
	+ Twitter: @anthropicai
	+ LinkedIn: linkedin.com/company/anthropic
	+ GitHub: github.com/anthropic

Join us on our mission to shape the future of AI.

In [46]:
stream_brochure("Anthropic", "https://anthropic.com")

# Anthropic Brochure
===============

## Company Overview
---------------

Welcome to Anthropic, the leading AI research and development company. We empower humans and machines to work together in pursuit of intelligent systems that can drive positive change.

Our mission is to harness the power of artificial intelligence to create a better future for all. With a team of renowned experts and researchers, we push the boundaries of what is possible with machine learning and natural language processing.

## Products/Services
-------------------

*   **AI Solutions**: We develop custom AI solutions for businesses and organizations looking to drive growth and innovation.
*   **Research Partnerships**: Collaborate with us on cutting-edge research projects that explore the frontiers of AI and machine learning.
*   **Training Data**: Access high-quality, diverse training data for your AI models, carefully curated by our expert researchers.

## Mission & Vision
-----------------

Our mission is to create intelligent systems that can help solve some of humanity's most pressing challenges. Our vision is a world where humans and machines work together to drive positive change.

We are committed to:

*   Advancing the field of AI research and development.
*   Delivering high-quality solutions that meet the needs of our clients.
*   Fostering collaboration and knowledge-sharing across industries and disciplines.

## Contact Information
--------------------

Stay in touch with us at:

### Website
https://anthropic.com

### Email
[info@anthropic.com](mailto:info@anthropic.com)

### Phone
+1 555 555 5555

We're always here to help. Reach out and discover how Anthropic can empower your organization's AI journey.

---

Feel free to ask me any questions you'd like.

In [47]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("Anthropic", "https://anthropic.com")

# Anthropic Brochure

## Company Overview

Anthropic is a cutting-edge technology firm dedicated to harnessing the power of artificial intelligence and machine learning. Our mission is to empower organizations with innovative solutions that drive business growth, improve efficiency, and enhance decision-making.

At Anthropic, we believe in leveraging AI to unlock human potential. We are passionate about creating technology that makes a meaningful impact on people's lives.

## Products/Services

We offer a range of products and services designed to help businesses succeed in today's fast-paced digital landscape.

* **AI-powered Chatbots**: Build conversational interfaces that engage customers, provide support, and drive conversions.
* **Machine Learning Solutions**: Develop predictive models that optimize operations, enhance customer experiences, and uncover new insights.
* **Custom AI Development**: Partner with us to create tailored solutions that address your unique business needs.
* **Training and Consulting**: Stay ahead of the curve with our expert guidance on AI strategy, implementation, and best practices.

## Mission & Vision

Our mission is to empower organizations with cutting-edge AI technology that drives real results. Our vision is a world where humans and machines work together to create a brighter future for all.

* **Empower businesses**: Help organizations unlock their full potential through innovative AI solutions.
* **Drive innovation**: Continuously push the boundaries of what's possible with AI and machine learning.
* **Foster collaboration**: Build strong relationships with clients, partners, and stakeholders who share our passion for AI.

## Contact Information

Ready to explore how Anthropic can help your business thrive? Get in touch with us today!

* Website: https://anthropic.com
* Phone: +1 (555) 123-4567
* Email: [info@anthropic.com](mailto:info@anthropic.com)
* Address: 123 Main St, San Francisco, CA 94105

Stay ahead of the curve with Anthropic. Join our journey and discover a brighter future for all.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>